In [1]:
LANGUAGE = "en"

In [2]:
import os
import glob
import pickle
from pathlib import Path

from tqdm import tqdm

In [3]:
!pip install -qU allophant numpy==1.26.4 pandas==1.5.3 # required for allophant to work  Ok.

## Lectura de datos

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
DATA_ZIP = f"/content/drive/MyDrive/psaha_2025_adhoc/data/{LANGUAGE}_sr16000.zip"
DATA_DIR = "input_audios"

In [6]:
if not Path(DATA_DIR).exists():
    !mkdir -p {DATA_DIR}

In [7]:
# If dir is empty:
if len(os.listdir(DATA_DIR)) == 0:
    !unzip -q "{DATA_ZIP}" -d "{DATA_DIR}"

In [8]:
data = []

audio_paths = glob.glob(os.path.join(f"{DATA_DIR}/output/words_wav/{LANGUAGE}/sr16000", "*.wav"))
print(f"Found {len(audio_paths)} audio files")

Found 153121 audio files


## RUN

In [9]:
# NOTE have to remove all appearances of summaryWriter in source code because of bug!!!
# in: /usr/local/lib/python3.11/dist-packages/allophant/estimator.py
# L38, 431, 523, 558, 1010

from allophant.estimator import Estimator

In [10]:
device = "cuda"
model, attribute_indexer = Estimator.restore("kgnlp/allophant", device=device)
supported_features = attribute_indexer.feature_names
# The phonetic feature categories supported by the model, including "phonemes"
print(supported_features)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


allophant.pt:   0%|          | 0.00/1.29G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/allophant/phonetic_features.py:1114: LanguageMappingWarning: Remapped some languages to a variant within the same macro language: {'swa': 'swh', 'est': 'ekk'}
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/allophant/phonetic_features.py:1184: SingletonFeatureWarning: Only one feature variant found in ['tone']
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

['stress', 'syllabic', 'short', 'long', 'consonantal', 'sonorant', 'continuant', 'delayedRelease', 'approximant', 'tap', 'trill', 'nasal', 'lateral', 'labial', 'round', 'labiodental', 'coronal', 'anterior', 'distributed', 'strident', 'dorsal', 'high', 'low', 'front', 'back', 'tense', 'retractedTongueRoot', 'advancedTongueRoot', 'periodicGlottalSource', 'epilaryngealSource', 'spreadGlottis', 'constrictedGlottis', 'fortis', 'raisedLarynxEjective', 'loweredLarynxImplosive', 'click', 'phoneme']


In [11]:
import tomllib
import importlib.resources as pkg_resources
import allophant.package_data

# Read the TOML file from the package
with pkg_resources.files(allophant.package_data).joinpath('default_config.toml').open('rb') as f:
    config = tomllib.load(f)

# consider all languages:
inventory = attribute_indexer.phoneme_inventory(config['data']["languages"])
print(inventory)

['a', 'b', 'bː', 'bʱ', 'd', 'dː', 'dʑ', 'e', 'ɛ', 'f', 'ɡ', 'ɡː', 'h', 'i', 'k', 'kː', 'kʰ', 'kʰː', 'l', 'm', 'n', 'ŋ', 'o', 'ɔ', 'p', 'pː', 's', 'ʃ', 'ʃː', 't', 'tː', 'tɕ', 'tʰ', 'tʰː', 'u', 'd̪', 'ə', 'j', 'ʎ', 'ɲ', 'r', 'ɾ', 't̪', 'ui̯', 'w', 'z', 'aː', 'au', 'c', 'ɛː', 'ɛu', 'ɦ', 'iː', 'ɪ', 'ɟ', 'oː', 'ou', 'r̝', 'ts', 't̠ʃ', 'uː', 'v', 'x', 'ʒ', 'ɑ', 'ç', 'e̞', 'lʲ', 'nʲ', 'ø̞', 'o̞', 'tʲ', 'ɯ', 'y', 'æʌ̯', 'ɒ', 'b̥', 'd̥', 'd̥z̥', 'ɡ̊', 'œ', 'oʌ̯', 'ʁ̞', 'ʋ', 'ʌ', 'aɪ', 'aʊ', 'øː', 'ɔɪ', 'ʊ', 'ɐ', 'ð', 'ɣ', 'n̪', 'θ', 'æ', 'd̠ʒ', 'eɪ̯', 'ɚː', 'iɪ', 'oʊ', 'ɹ', 'ø', 'ai̯', 'au̯', 'ei̯', 'eu̯', 'oi̯', 's̺', 's̪̻', 't̺s̺', 't̪̻s̪̻', 'æː', 'e̯', 'eː', 'ŋː', 'yː', 'a̟', 'æ̃', 'ɡ̟', 'k̟', 'õ', 'œ̃', 'ʀ', 'ɑː', 'bʲ', 'ə̯', 'k̟ʰ', 'mʲ', 'n̥ʲ', 'ɸ̃', 's̻', 'βʲ', 'ãː', 'd̪ː', 'ɖ', 'ẽː', 'ɜ', 'ɜ̃', 'ĩː', 'õː', 'ɔ̤', 'pʰ', 'q', 'ʂ', 't̪ː', 't̪ʰ', 't̪ʰː', 'ʈ', 'ʈʰ', 'ũː', 'ʊ̃', 'χ', 'cç', 'cçː', 'dzː', 'ɟʝ', 'ɟʝː', 'tsː', 't̠ʃː', 'ʔ', 'd̪z̪ː', 'd̠ʒː', 'ɛi̯', 'ʎː', 'ɲː', 's̪', 't̪s̪ː', 'u

In [12]:
import torch
import torchaudio
from allophant.dataset_processing import Batch
from allophant import predictions

# Create a feature mapping for your inventory and CTC decoders for the desired feature set
inventory_indexer = attribute_indexer.attributes.subset(inventory)
ctc_decoders = predictions.feature_decoders(inventory_indexer, feature_names=supported_features)

def process_wav(wav_path):
    # Load an audio file and resample the first channel to the sample rate used by the model
    audio, sample_rate = torchaudio.load(wav_path)
    audio = torchaudio.functional.resample(audio[:1], sample_rate, model.sample_rate)

    # Construct a batch of 0-padded single channel audio, lengths and language IDs
    # Language ID can be 0 for inference
    batch = Batch(audio, torch.tensor([audio.shape[1]]), torch.zeros(1))
    model_outputs = model.predict(
        batch.to(device),
        attribute_indexer.composition_feature_matrix(inventory).to(device)
    )
    # return model_outputs
    feature_name = "phoneme"
    decoder = ctc_decoders[feature_name]
    decoded = decoder(model_outputs.outputs[feature_name].transpose(1, 0), model_outputs.lengths)
    # Print the feature name and values for each utterance in the batch
    for [hypothesis] in decoded:
        # NOTE: token indices are offset by one due to the <BLANK> token used during decoding
        recognized = inventory_indexer.feature_values(feature_name, hypothesis.tokens - 1)
    return recognized

In [13]:
# files_example = sorted([f for f in audio_paths if '/estribillo_' in f])
# files_example = sorted(audio_paths) #[:100]
# files_example = sorted(audio_paths)

results = {}
for f in tqdm(audio_paths):
    results[f] = process_wav(f)

100%|██████████| 153121/153121 [1:37:17<00:00, 26.23it/s]


In [14]:
outfile = f"/content/drive/MyDrive/psaha_2025_adhoc/output/allophant_{LANGUAGE}_sr16000_ipa.pkl"
Path("/content/drive/MyDrive/psaha_2025_adhoc/output").mkdir(exist_ok=True, parents=True)

with open(outfile, "wb") as f:
    pickle.dump(results, f)
print(f"Data saved to {outfile}")

Data saved to /content/drive/MyDrive/psaha_2025_adhoc/output/allophant_en_sr16000_ipa.pkl


-----------------

In [ ]:
from IPython.display import Audio, display
for i, (f, res) in enumerate(results.items()):
    # display(Audio(f, rate=16000))
    print(Path(f).name)
    print(res)
    print("-"*100)
    if i > 100:
        break

baratija_es023913_es.Castellano.wav
['b', 'a', 'ɾ', 'a', 't̪', 'i', 'x', 'a']
----------------------------------------------------------------------------------------------------
veterinaria_es198809_es.Argentina.wav
['v', 'e', 't̪', 'e', 'ɾ', 'ɪ', 'n', 'a', 'ɾ', 'i', 'a']
----------------------------------------------------------------------------------------------------
palangana_es141463_es.Argentina.wav
['a', 'l', 'ə', 'ŋ', 'ɠ', 'ɑ', 'n', 'a']
----------------------------------------------------------------------------------------------------
perpetrar_es147854_es.Argentina.wav
['ə', 'p', 'ə', 't̪', 'ɾ', 'a']
----------------------------------------------------------------------------------------------------
sesgo_es177518_es.Mexico.wav
['s̪̻', 'e', 'z̪', 'ɡ', 'o']
----------------------------------------------------------------------------------------------------
campanilla_es032623_es.Argentina.wav
['e̞', 'm', 'p', 'a', 'n', 'i', 'zʲ', 'e̞']
--------------------------------------